In [ ]:
# Imports
import pandas as pd
from random import randint
from src import *
from src.simulator import SIMULATOR

In [ ]:
sim = SIMULATOR()

# --------------------------------------------
#               KERNEL CONFIGURATION
# --------------------------------------------
kernel_path = './kernels/mac_16b_test/'
kernel_number = 1 
column_usage = [True, False] 
nInstrPerCol = 8 
imem_add_start = 0 
srf_spm_addres = 0 
version=""

sim.kernel_config(column_usage, nInstrPerCol, imem_add_start, srf_spm_addres, kernel_number)

In [ ]:
# --------------------------------------------
#                LOAD SPM DATA
# --------------------------------------------
# Load vector A
vector_A = [randint(1, 1000) for i in range(SPM_NWORDS)]
nline = 1
sim.setSPMLine(nline, vector_A.copy())
# Load vector B
vector_B = [randint(1, 1000) for i in range(SPM_NWORDS)]
nline = 2
sim.setSPMLine(nline, vector_B.copy())
# Load vector C
vector_C = [randint(1, 1000) for i in range(SPM_NWORDS)]
nline = 3
sim.setSPMLine(nline, vector_C.copy())

sim.displaySPMLine(1)
sim.displaySPMLine(2)
sim.displaySPMLine(3)

In [ ]:
# --------------------------------------------
#              COMPILE ASM TO HEX
# --------------------------------------------
sim.compileAsmToHex(kernel_path, kernel_number, version=version)

In [ ]:
# --------------------------------------------
#                 LOAD KERNEL
# --------------------------------------------

# This needs the hex instructions, if you don't provide them, generate then compiling the asm
sim.kernel_load(kernel_path, version=version + "_autogen", kernel_number=kernel_number)

# --------------------------------------------
#               SIMULATE EXECUTION
# --------------------------------------------
show_lcu = []
show_srf = []
show_lsu = []
show_rcs = [[],[],[],[]]
show_mxcu = []
display_ops = [show_lcu, show_lsu, show_mxcu, show_rcs, show_srf]

sim.run(kernel_number, display_ops=display_ops)

Let's check that we have the correct output in the SPM line just by looking at it.

In [ ]:
sim.displaySPMLine(1)
sim.displaySPMLine(2)
sim.displaySPMLine(3)

We can check it more rigorously. We can define our function in python and check that the output matches the CGRA output.

In [ ]:
def mac32b (in_A, in_B, in_C, out):
    for i in range(len(in_A)):
        out[i] = in_A[i] * in_B[i] + in_C[i]

def mac16b (in_A, in_B, in_C, out):
    for i in range(len(in_A)):
        val1_high = (in_A[i] >> 16) & 0xFFFF  
        val1_low = in_A[i] & 0xFFFF           
        val2_high = (in_B[i] >> 16) & 0xFFFF  
        val2_low = in_B[i] & 0xFFFF
        val3_high = (in_C[i] >> 16) & 0xFFFF  
        val3_low = in_C[i] & 0xFFFF  

        high_res = (val1_high * val2_high) & 0xFFFF
        low_res = (val1_low * val2_low) & 0xFFFF

        high_res = (high_res + val3_high) & 0xFFFF
        low_res = (low_res + val3_low) & 0xFFFF

        out[i] = (high_res << 16) | low_res
        

In [ ]:
sim.displaySPMLine(3)
vwr2a_res = sim.getSPMLine(3)
expected_output = np.zeros(len(vector_A))
mac16b(vector_A, vector_B, vector_C, expected_output)
errors_idx = []
for i in range(len(vector_A)):
    if expected_output[i] != vwr2a_res[i]:
        if len(errors_idx) == 0:
            print("Oops, something went wrong.")
        errors_idx.append(i)
        if len(errors_idx) < 5:
            print("CGRA[" + str(i) + "]: " + str(vwr2a_res[i]) + " != " + str(expected_output[i]))
if len(errors_idx) == 0:
    print("The result is correct!")
else:
    print("There are " + str(len(errors_idx)) + " errors.")


Now it's your time to play!

In [ ]:
#Let's do it!